# REST API Tour of Identity Cloud Service 17.4.2

When you finish this, you will have a basic idea of how to use REST API interface of IDCS

You will provide necessary details of your IDCS tenant; authenticate,generate, and inspect Access Token (AT); and finally use the AT to call IDCS User and Reporting API's.

**After this assignment you will be able to:**

- Explore and generate IDCS Access Token using 2-legged OAuth2 Authorization flow
- Call IDCS User Management API's
- Call IDCS Audit API's

Let's get started!
___

## 1 - Discovery

First let's gather information about your IDCS Tenant

Run the following cell. Provide the following information and **Click** on **`Discover IDCS Config`** Button
- Tenant name
- Domain Name

Then you will discover the IDCS End Points automatically.

Explore by expanding the Discovered **`IDCS Configuration`**

In [ ]:
import pandas as pd, numpy as np, datetime as dt, matplotlib.pyplot as plt
import requests, json
from ipywidgets import widgets, Layout
from IPython.display import display

baseurl = None
config = None
tenant = widgets.Text()
tenantD = widgets.HBox([widgets.Label(value="Tenant Name"), tenant])
domain = widgets.Text(value="identity.oraclecloud.com")
domainD = widgets.HBox([widgets.Label(value="Domain Name"), domain])
port = widgets.Text(value="443")
portD = widgets.HBox([widgets.Label(value="Port"), port])
recordenv = widgets.Button(description='Discover IDCS Config', button_style='info', tooltip='Get IDCS Confgiuration',)                     
display(tenantD)
display(domainD)
display(portD)
display(recordenv)

def handle_submit(sender):
    global config, baseurl
    baseurl = 'https://' + tenant.value + '.' + domain.value + ':' + port.value
    configurl =  baseurl + '/.well-known/idcs-configuration'
    response = requests.get(configurl).text
    config = json.loads(response)
    configD = widgets.Accordion(children=[widgets.Textarea(value=json.dumps(config, indent=4, sort_keys=True), 
                                                             layout=Layout(width='100%', height='800px'))])
    configD.set_title(0, 'IDCS Configuration')
    configD.selected_index = None
    display(configD)
    
recordenv.on_click(handle_submit)

The Discovery End-Point was public and so didn't require any Access Token or other form of authentication

___

## 2 - Access Token (2-legged OAuth)

Next We will get an OAuth Access Token  using 2-legged OAuth2 Authorization flow.

Check out the following diagram of 2-legged flow -

![OAuth2 2-Legged Diagram](resources/oauth2legged.jpg)

Some minimal setup in IDCS is needed for the flow. 

- Login to your [IDCS Admin Console]() using Administrator credentials
- Create an **App** that grants access to IDCS protected REST API's. 
- Record the `Client ID` and `Client Secret` from the App Configuration

Follow the video below to create your App in IDCS.

In [ ]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/A2LiNJRRINk?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Now it's time to store the recorded `Client ID` and `Client Secret`. They will be used duing API call.

In [ ]:
from getpass import getpass 
#Lets get the App's Client ID and Client secret. They will be needed for proceted API calls
clientId = input('Client ID: ')
clientSecret = getpass(prompt='Client Secret: ')

For 2-legged OAuth flow, you will need to provide User credentials. Gather the credentials below.

In [ ]:
#Get Username and Password
userName = input('Login ID: ')
userPwd = getpass(prompt='Password: ')

Having collected all the necessary data, you can now generate **Access Token** from IDCS.

The steps are -
- Find the Token End-Point URL from IDCS Discovery Configuation
- Specify the grant type as `password`
- Pass the `Client ID` and `Client Secret` in base-64 encoded format as **HTTP Basic Authentication** header.  
    > The format of the data is -
    **<font color="blue">b64encode(Client ID:Client Secret)</font>**
- Make an **HTTP POST** call to the Token End-Point URL
    - Include the `Authentication` Header
    - POST data should contain - `Grant Type`, `User ID`, `User Password` and `Scope`
- The POST response will contain the issued **Access Token**
  
View the **AT** from displayed Output   

In [ ]:
import base64, urllib.parse

#Find the OAuth2 Token Endpoint from Discovery Config
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "password" # Token grant_type for 2-legged flow
scope = "urn:opc:idm:__myscopes__"

#Token Endpoint needs the App Creds in base64 encoded Header
basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret, 'utf-8')) 
reqdata = {'grant_type': grant_type, 'username': userName, 'password': userPwd, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders)
accesstoken = response.json()["access_token"]
accesstokenD = widgets.Accordion(children=[widgets.Textarea(value=accesstoken, layout=Layout(width='100%', height='400px'))])
accesstokenD.set_title(0, 'Access Token')
accesstokenD.selected_index = None
display(accesstokenD)

Let's now examine the Access Token thus generated. Expand **Inspect Token** from the following cell's output and study the decoded token. Specifically observe the following attributes -
> <font color="blue">tok_type, aud, clientAppRoles, userAppRoles, scope, client_name, tenant</font>

An `Access Token` can be used to call API's that fall under the **scope** values the token was issued with.


In [ ]:
import jwt

jwtoken = jwt.decode(accesstoken, verify=False)

jwtokenD = widgets.Accordion(children=[widgets.Textarea(value=json.dumps(jwtoken, indent=1), layout=Layout(width='100%', height='600px'))])
jwtokenD.set_title(0, 'Inspect Token')
jwtokenD.selected_index = None
display(jwtokenD)


___

## 3 - User Management

Armed with Access Token having **Scopes** that grant access to **User Administrator** level Admin API's, let's now get all the users currently in **IDCS**.

Run the following cell to display IDCS Users with their **Email Address** and internal **ID** values

> Notice that the User REST API call contains an **Authorization** Header where the **AT** is passed as a **Bearer Token**

In [ ]:
userurl = baseurl + "/admin/v1/Users"

uheaders = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
userlist = requests.get(userurl, headers = uheaders).json()["Resources"]
userDF = pd.DataFrame(userlist)
userDispList = [userDF["userName"], 
                userDF.emails.apply(lambda x: x[0]["value"] if x[0]["primary"] == True else x[1]["value"]), 
                userDF["id"]]
userDispDF = pd.concat(userDispList, axis=1)

HTML(userDispDF.to_html())


___

## 4 - Analytics and Auditing

If you have created the App exactly as shown in the Video above, the Access Token will also have access to **Audit Administrator** level Admin API's. That will enable you to query the **`Audit Events`** from IDCS and retrive data for **Analytics**.

Let's query IDCS **Audit Events** API to retrieve **<font color="blue">Login Success</font>** and **<font color="blue">Login Failure<font>** Events. Those 2 events are passed to the API as **filter** conditions, since the Audit API fetches all possible user and admin level events in IDCS.

In [ ]:
auditurl = baseurl + '/admin/v1/AuditEvents?filter=(eventId co \"sso.authentication.failure\" or eventId co \"sso.session.create.success\")&count=32000'

audheaders = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
audevents = requests.get(auditurl, headers = audheaders).json()["Resources"]
audeventsDF = pd.DataFrame(audevents)
audeventsDF["timestamp"] = pd.to_datetime(audeventsDF["timestamp"])

modDF = audeventsDF

Now that the Login data has been retrived, let's plot to see the **Login Trend** of past 30 days 

In [ ]:
NoOfDays = 30

pltDF = modDF[modDF["timestamp"] >= dt.datetime.now() - dt.timedelta(days=NoOfDays)]
logindata = pltDF.groupby([pltDF["timestamp"].dt.date, 'eventId'])['ecId'].count()

logindata.unstack().plot(kind="bar")
plt.show()

___